# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [9]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?url=RJ&tipo_interessado=Estado'
response = requests.get(url)
json = response.json()
df = pd.DataFrame(json['items'])
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,12070,Estado,Sergipe,28,SE,None,Arquivado,17944.001597/2007-37,2009-07-01T03:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento sustentável,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,6.720000e+07,0,0,21/07/2009
1,10836,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001297/2014-87,2015-11-13T17:12:11Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.900000e+08,1,0,04/02/2016
2,10844,Estado,Santa Catarina,42,SC,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001299/2012-12,2012-11-26T02:00:00Z,Operação contratual externa (com garantia da U...,Reestruturação e recomposição do principal de ...,Instituição Financeira Internacional,Bank of America Merrill Lynch,Dólar dos EUA,7.264416e+08,1,0,03/01/2013
3,10859,Estado,Bahia,29,BA,None,Deferido,17944.001302/2013-71,2013-10-11T03:00:00Z,Operação contratual interna,PAC 2 - Mobilidade Grandes Cidades,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4.000000e+08,1,0,18/10/2013
4,12113,Estado,Sergipe,28,SE,None,Deferido,17944.001606/2009-51,2010-02-01T02:00:00Z,Operação contratual interna,PSI - Programa de Sustentação do Investimento,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4.226400e+06,1,0,05/03/2010


In [16]:
# 2) Seu código aqui
df['status'].value_counts()[0:3]

Encaminhado à PGFN com manifestação técnica favorável    446
Deferido                                                 441
Arquivado a pedido                                        73
Name: status, dtype: int64

In [23]:
# 3) Seu código aqui
df['ano_status'] = df['data_status'].str.split('/').map(lambda x: x[2])
df[['id_pleito', 'data_status', 'ano_status']].head(10)

,id_pleito,data_status,ano_status
0,12070,21/07/2009,2009
1,10836,04/02/2016,2016
2,10844,03/01/2013,2013
3,10859,18/10/2013,2013
4,12113,05/03/2010,2010
5,10896,17/05/2013,2013
6,9723,22/10/2009,2009
7,16571,03/10/2008,2008
8,10243,19/01/2004,2004
9,10248,19/01/2004,2004


In [24]:
# 4) Seu código aqui
df['ano_status'].value_counts()

2013    122
2009    105
2010     97
2012     90
2014     70
2008     64
2006     57
2002     56
2011     49
2022     43
2017     43
2018     42
2004     40
2021     40
2005     37
2016     34
2019     25
2020     23
2007     20
2015     17
2023      8
2003      6
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [26]:
#1) Seu código aqui
def ler_dados_api(uf: str, interessado: str) -> pd.DataFrame:
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?url={uf}&tipo_interessado={interessado}'
    response = requests.get(url)
    json = response.json()
    return pd.DataFrame(json['items'])

In [29]:
# 2) Seu código aqui
df_minas = ler_dados_api('MG', 'Estado')
df_minas['status'].value_counts()['Arquivado por decurso de prazo']

32

In [41]:
# 3) Seu código aqui
df_bahia = ler_dados_api('BA', 'Município')
deferido = (df_bahia['uf'] == 'BA') & (df_bahia['status'] == 'Deferido')
df_bahia[deferido]['interessado'].value_counts()[0:1]

Luís Eduardo Magalhães    4
Name: interessado, dtype: int64

In [44]:
# 4) Seu código aqui
nome_arquivo = 'solicitacoes_estado_bahia.csv'
df_estado_bahia = ler_dados_api('BA', 'Estado')
df_estado_bahia.to_csv(nome_arquivo, index = False)